## Библиотеки проекта

Подключим список библиотек, которые используются в данном проекте. Импортируем библиотеки *pandas*, *pymystem3* и *collections* в наш проект. 

In [1]:
!pip install pymystem3

# Импорт библиотеки pandas
import pandas as pd

# Импортируем библиотеку pymysteam3
from pymystem3 import Mystem

# Импортируем библиотеку collections
from collections import Counter

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Предстоит ответить на четыре вопроса:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

### Оглавление

- [Этап 1. Получение данных](#s1)


- [Этап 2. Предобработка данных](#s2)
  - [Наименования столбцов](#s2-1)
  - [Обработка пропусков](#s2-2)
  - [Замена типа данных](#s2-3)
  - [Обработка дубликатов](#s2-4)
  - [Лемматизация](#s2-5)
    - [Леммы для целей кредита. Первый вариант – множество](#s2-5-1)
    - [Леммы для целей кредита. Второй вариант – словарь](#s2-5-2)
  - [Категоризация](#s2-6)
    - [По детям](#s2-6-1)
    - [По доходу](#s2-6-2)
    - [По целям](#s2-6-3)
    - [По задолженности](#s2-6-4)
    
    
- [Этап 3. Ответы на вопросы](#s3)
  - [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#s3-1)
  - [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#s3-2)
  - [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#s3-3)
  - [Как разные цели кредита влияют на его возврат в срок?](#s3-4)
  
  
- [Этап 4. Вывод](#s4)

# Этап 1. Получение данных <a id="s1"></a>

#### Получим и сделаем первичные выводы о предоставленных данных заказчиком

Прочитаем файл *data.csv* с входными данными и сохраним его в переменной *data*. Выведем первых 5 строк таблицы *data*, чтобы визуально ознакомиться с данными.

In [2]:
# Чтение файла с данными с сохранением в data
data = pd.read_csv('datasets/data.csv')
#data = pd.read_csv('/datasets/data.csv')

# Получение первых 5 строк таблицы data методом head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Давайте получим общую информация о данных таблицы *data* методом info(), а также ознакомимся с наличием пропусков и типами данных.

In [3]:
# Получение общей информации о данных в таблице data методом info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице *data* – 12 столбцов:
- **строковые** – 5 столбцов;
- **целочисленные** – 5 столбцов;
- **с плавающей точкой** – 2 столбца.

Подробно разберём, какие столбцы в таблице *data* и какую информацию они содержат:

**object:**
- *education* — уровень образования клиента;
- *family_status* — семейное положение;
- *gender* — пол клиента;
- *income_type* — тип занятости;
- *purpose* — цель получения кредита.


**integer:**
- *children* — количество детей в семье;
- *dob_years* — возраст клиента в годах;
- *education_id* — идентификатор уровня образования;
- *family_status_id* — идентификатор семейного положения;
- *debt* — имел ли задолженность по возврату кредитов;


**float:**
- *days_employed* — общий трудовой стаж в днях;
- *total_income* — ежемесячный доход.


Количество значений в столбцах различается, но не значительно.
Выпадают из общего списка 2 столбца:
- общий трудовой стаж в днях;
- ежемесячный доход.

и оба типа *float*

## Вывод

Строки таблицы содержат в себе информацию о клиентах, которые брали кредитные средства под определенные нужды. Также имеются данные о возрасте клиента, образовании, семейном положении, количестве детей, трудовом стаже, типе занятости, ежемесячном доходе и его принадлежность к определенному гендору.

Для подготовки данных к анализу необходимо разобраться со следующими пунктами:
- **days_employed (общий трудовой стаж в днях)** – наблюдаются отрицательные значения, а также очень большие положительные. Необходимо понять природу их происхождения, выявить закономерность и аномальность этих данных;
- **данные с различным регистром** – привести значения в единый регистр для дальнейшего анализа;
- **поиск дублей** – так как отсутствует уникальный идентификатор пользователя, по-которому мы бы смогли сделать проверку на дубли, есть риск очистить схожие данные.
- **пропуски и 0 значения** – проверить данные на пропуски и отрицательные значения (в столбцах, где "-" выглядит аномальным). Для одних восстановить средними или медианными значениями, чтобы не потерять репрезентативнось данных, а для других сделать положительными;
- **числа с плавающей точкой** – тип *float* привести в целочисленный вид данных *integer*;
- **purpose (цель получения кредита)** – лемматизировать значения в нормальную словоформу.
- Для проверки рабочих гепотез особенно важны столбцы **debt**, **children**, **family_status**, **total_income** и **purpose**. Поэтому почти все колонки категоризируем, для формирования визуально понятных выводов по категориям.

# Этап 2. Предобработка данных<a id="s2"></a>

#### Обработаем пропуски, изменим тип данных, проверим данные на наличие дубликатов, лемматизируем цели получения кредита и категоризируем данные.

## Наименования столбцов<a id="s2-1"></a>

Проверим названия столбцов на возможные пробелы, чтобы в дальнейшем не было проблем к их обращению. Получим список названий столбцов в нашей исследуемой таблицы с данными *data*.

In [4]:
# Получение перечня названий столбцов таблицы data
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

## Вывод

В названиях колонок отсутствуют пробелы, что позволит без каких-либо проблем обращаться к данным через их нименование.

## Обработка пропусков<a id="s2-2"></a>

Проверим общее кол-во пропусков в таблице **data методом** **isnull().sum()**

In [5]:
# Cуммарное количество пропусков, выявленных методом isnull() в таблице data
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что в столбцах **days_employed** и **days_employed** имеется одинаковое количество пропусков.
Попробуем исследовать эти данные, отфильтровав таблицу **data** по столбцам **days_employed** и **days_employed**, в которых встречаются пропуски и запишем в переменную **data_null**.

In [6]:
# Фильтр таблицы data по столбцам days_employed и days_employed c isnull() = True и записью в переменную data_null
data_null = data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]

# Получение общей информации о данных в таблице data_null
data_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Количество пропусков – **2174** в столцах таблицы **data** соответствует количеству строк **2174** таблицы **data_null**. Можно сделать промежуточный вывод о том, что эти данные получали из одного источника или в одно время, когда информация о стаже и доходе клиента не собиралась.

Расчитаем процент пропусков от общего количества данных: возьмем частное от длины **data_null** и **data** и выведем их процент. А также посчитаем кол-во клиентов с задолженностю по кредиту, взяв сумму по столбцу **debt** в **data_null**.

In [7]:
# Вывод процента пропусков от общего количества данных
print('Процент пропусков от общего количества данных: {:.0%}'.format(len(data_null) / len(data)))

# Подсчет количества клиентов в пропусках, у которых были задолженности возврата кредита 'debt'
print('Кол-во клиентов в пропусках с задолженностью по кредиту:', data_null['debt'].sum())

Процент пропусков от общего количества данных: 10%
Кол-во клиентов в пропусках с задолженностью по кредиту: 170


Достаточно большой процент пропущенных значений, чтобы пренебречь этими данными. Также в пропусках имеются важные данные о задолженности возврата кредита.

Проработаем пропуски по ежемесячному доходу и заполним их.

In [8]:
# Вывод минимального значения дохода из столбца total_income методом min()
print('Минимальное значение дохода: {:.0f}'.format(data['total_income'].min()))

# Вывод максимального значения дохода из столбца total_income методом max()
print('Макимальное значение дохода: {:.0f}'.format(data['total_income'].max()))

Минимальное значение дохода: 20667
Макимальное значение дохода: 2265604


Так как отрицательных значений не имеем, то мы можем использовать данные колонки **total_income** для расчета средних значений. 

Расчет среднего значения о ежемесячном доходе будет репрезентативнее с использованием **median()** колонки *total_income*. Потому как среднее значение выявленное методом *mean()* будет искажать реальные данные за счет огромного значения **2 265 604**.

Для более достоверного расчета среднего зачения ежемесячного дохода для пропусков, расчитаем медианы значений по типу занятости в колонке **'income_type'** таблицы **data** и запишем их в словарь **total_income_median** с парой **ключ: значение: {income_type: total_income.median()}**. 

Напишем функцию **income_filter**, которая будет отдавать значения взятые из списка **total_income_median** по ключу **income_type** из условия наличия пропуска в ячейки **total_income**. Далее, использую функцию **income_filter**, подставим значения в пропуск методом **apply()**

Переведем все значения в колонках с типом **object** в нижний регистр методом **str.lower()**, чтобы в дальнейшем было комфортно с ними работать, не потеряв лишние данные. Создавать дополнительную колонку не имеет смысла, так как это категориальные значения и необходимости держать такие данные в различном регистре – нет. При этом пропуски с этим типом данных отсутствуют.

In [9]:
# напишем цикл, в котором будем искать все колонки с типом данных object
# и изменять значения в нижний регистр у этих колонок

# создаем цикл в котором перебираем значения длины кол-ва_столбцов_в_data
# внутри цикла создадим условие проверки данных колонки равным object 
# если пересечения есть, то заменяем значения этой колонки на нижний регистр
for item in range(len(data.columns)):
    if data[data.columns[item]].dtypes == object:
        data[data.columns[item]] = data[data.columns[item]].str.lower()

In [10]:
# Словарь с медианными значениями total_income по сгрупперованной колонке income_type таблицы data
total_income_median = data.groupby('income_type')['total_income'].median().to_dict()

# Функция income_filter(), которая принимает один элемент серии, 
# проверяет значение total_income на пропуск методом pd.isna(), 
# если такое значение встречается, то возвращает значение из 
# словаря total_income_median, которое соответствует income_type в нем,
# если нет, то возвращает изначально переденнаое значение total_income
def income_filter(row):
    if pd.isna(row['total_income']):
        return total_income_median[row['income_type']]
    return row['total_income']

# Перезапишем значения для колонки total_income в таблице data, используя метод apply(),
# в который передадим нашу функцию income_filter и параметр axis=1
data['total_income'] = data.apply(income_filter, axis=1)


# Проверим кол-во пропусков после перезаписи
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Пропуски о ежемесячном доходе отсутствуют, перейдем к данным **days_employed** о стаже работы в днях. Выведем информацию о значениях столбца **days_employed** через метод **describe()**

In [11]:
# Вывод информации о данных столбца days_employed методом describe()
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Встречаются как положительные, так и отрицательные значения. Попроуем проанализировать положительные значения. Посчитаем среднеарифметическое положительных значений колонки **days_employed** методом **mean()** и поделим на 365 дней, чтобы увидеть предпологаемое значение стажа в годах.

In [12]:
# Посчитаем средний стаж в годах, на примере среднеарифметического положительного значения колонки days_employed, поделив его на 365
print('Средний стаж {:.0f} лет'.format((data[data['days_employed'] > 0]['days_employed'].mean())/365))

Средний стаж 1000 лет


In [13]:
# Получение первых 5 строк таблицы data с содержанием положительных значениях days_employed
data[data['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,f,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,не женат / не замужем,4,f,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,m,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,f,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


Подозрительно **большие числа для стажа в днях** и тем более в годах – **1000 лет**, значит у этих данных могла быть другая логика формирования. На практике это можно было бы уточнить у заказчика и сделать пересчет, если данные хранятся, допустим, в часах.

Также мы можем наблюдать некую корреляцию между данными с большим положительным числом в колонке со стажем **days_employed** и типом занятости **income_type**. Давайте проверим насколько эта связь близка. Посчитаем кол-во пересечений в значениях **days_employed > 0** и **income_type = 'пенсионер'** через метод *count()* и сравним кол-вом положительных значений в колонке **days_employed**.

In [14]:
# Посчитаем кол-во значений методом count() удовлетворяющих условию days_employed > 0 и income_type = 'пенсионер'
print('Кол-во значений "пенсионер":',data[(data['days_employed'] > 0) & 
                              (data['income_type'] == 'пенсионер')]['days_employed'].count())


# Всего значений days_employed > 0 – положительных значений стажа в таблице data
print('Всего значений с положительным стажем:', data[data['days_employed'] > 0]['days_employed'].count())

Кол-во значений "пенсионер": 3443
Всего значений с положительным стажем: 3445


Видна взаимосвязь положительных значений и типе занятости – "*пенсионер*" в колонке **days_employed** таблицы *data*. К сожалению вести дальнейшую работу с положительными значениями колонки **days_employed** нет смысла, в ввиду логической не корректности.

Разберем отрицательные данные в стобце **days_employed** и попробуем понять их природу появления.

Попробуем расчитать среднеарифметический стаж в годах из отрицательных значений методом *mean()*. Поделим данное значение на 365 и обернем в модуль **abs()**, чтобы получить положительное значение.

In [15]:
# Модуль среднего стаж в годах
print('Средний стаж в годах {:.1f}'.format(abs((data[data['days_employed'] < 0]['days_employed'].mean()) / 365)))

Средний стаж в годах 6.4


Данные близки к действительности, но и их природа появления, также не особо ясна, как и положительных значений. Данные о стаже нам не особо важны, так как они не коррелируют с условием поставленной задачи Бизнеса, поэтому с обработкой пропусков можем поступить одним из предложенных способов:
- оставить равным 0, чтобы при удалении пропусков строки остались и мы не потеряли часть важных данных;
- дозаполнить среднеарифметическим или медианым из отрицательных значений колонки *days_employed*;
- удалить колонку 'days_employed', посчитав ее "мусорной" для решения поставленных задач, в виду не корректности данных.

Давайте удалим колонку **days_employed** методом **drop()** с параметром **axis=1** из нашей таблицы **data**

In [16]:
# Удаление колонки days_employed методом drop() с параметром axis=1
data = data.drop('days_employed', axis=1)

# Проверим значения одной из важных колонок children 
data['children'].describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [17]:
print('Кол-во отрицательных значений:', data[ data['children'] == data['children'].min()]['children'].count())

Кол-во отрицательных значений: 47


Таких анных не так много, скорее всего какой-то сбой системы, давайте переведем отрицательные значения в положительные через модуль **abs()**

In [18]:
# Замена отрицательных значений на положительные в колонке children через модуль
data['children'] = abs(data['children'])

# Проверим пропуски в таблице data
data.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Вывод

1. Пустые значения свидетельствуют о том, что информация о **трудовом стаже** и **ежемесячном доходе** для некоторых клиентах **не доступна**. Причины могут быть разные, но эти **данные отсутствуют у одних и тех же клиентов**. Скорее всего источник получения данных чем-то отличается и эта информация не передается в общую систему данных Заказчика.


2. **Пропуски данных составили 10%** от общего кол-ва данных и это достаточно много, чтобы мы могли такими данными пренебречь. При этом **в них хранится информация** о важном показателе – **факте задолженности по кредиту**.


3. Для заполнения пропусков о **ежемесячном доходе** мы вычислили **медианные значения** сгруппированных данных по типу занятости, которые являются более репрезентативными, чем среднеарифметическое значение, потому как этот метод **не подвержен влиянию больших цифр**.


4. Пропуски в колонке **days_employed**, было решено удалить в виду неизвестной природы формирования значений, а также из-за отсутсвия связей с поставленными задачами Заказчиком. Хотя часть значений и коррелируют со столбцом **income_type**.


5. В колонке **children** наблюдалось небольшое кол-во отрицательных значений, скорее всего это какая-то ошибка при внесении данных. Мы изменили отрицательные значения на положительные.

## Замена типа данных<a id="s2-3"></a>

Исходя из имеющихся данных о типах столбцов мы можем изменить тип **total_income** с *float64* на **int64**, здесь достаточно большие значения, в которых дробной частью можно пренебречь.

После замены типа данных, запросим методом *info()* получившуюся информацию. А так же выведем 10 строк методом *head()*, чтобы увидеть визуальные изменения в данных. 

In [19]:
# Замена типа данных столбца total_income таблицы data на Integer
data['total_income'] = data['total_income'].astype('int')

In [20]:
# Просмотрим общую информацию о таблице data, чтобы удостовериться о замене типа данных 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


In [21]:
# Выведем первые 5 строк таблицы data методом head()
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу


## Вывод

При замене типа мы логически учли принятые нами данные и провели изменения в **total_income** *float64* => **int64**, здесь достаточно большие значения, в которых дробной частью можно пренебречь

Изменеие типа данных уменьшило размер данных и упростило визуальную обработку.

## Обработка дубликатов<a id="s2-4"></a>

Давайте проверим кол-во дубликатов в нашей таблице *data*, используя метод **duplicated()** с подсчетом суммы значений – **sum()**. Искать дубликаты будем в рамках всех столбцов, т.к. у нас нет колонки с уникальным идентефикатором данных, чтобы проверку сделать именно по нему. Если такие значения найдем, то удаляем и проверяем, все ли удалилось.

In [22]:
# Проверим кол-во дубликатов в нашей таблице *data* методом duplicated().sum()
data.duplicated().sum()

71

In [23]:
# Удаляем дубликаты из нашей таблицы с перезаписью порядкового индекса
data = data.drop_duplicates().reset_index(drop = True)

# Перепроверим кол-во дублей в нашей таблице data
data.duplicated().sum()

0

## Вывод

Мы удалили 71 дубликат, чтобы наши данные стали чище. Удаление дубликатов произвели только по полностью совпадающим строкам.

**НО!** На самом деле, если у входных данных нет идентификаторов, которые реально могут нести в себе унакальность каждой строки, мы не должны избавляться от таких данных. Даже при таком кол-ве колонок в таблице, могут содержаться данные с повторяющимися значениями, соответствующим разным клиентам.

## Лемматизация<a id="s2-5"></a>

Мы имеем колонку **purpose** с целью получения кредита, мы можем лемматизировать ее значения в новый столбец **purpose_lem**. Импортируем библиотеку **pymystem3**, т.к. она лучше всех подходит **для лемматизации предложений на русском языке**. Нам необходимы **правильные словоформы**, чтобы в дальнейшем была возможность верно категоризировать эти данные. 

In [24]:
# Применим к переменной m метод Mystem()
m = Mystem()


# Создадим функцию которая будет принимать один элемент серии, 
# а возвращать строку с разделенными значениями через пробел из
# лемматизированного списока значений ключа purpose, этого элемента серии.
def lemming(row):

    return ' '.join(m.lemmatize(row['purpose']))



# Создадим новый столбец purpose_lem с помощю apply(), 
# в которую помещаем нашу функцию lemming и параметр axis=1.
# В purpose_lem будем хранить строку с нормальными словоформами из purpose
data['purpose_lem'] = data.apply(lemming, axis=1)


# Выведем первые 5 строк
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem
0,1,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


### Первый вариант формирования категорий через леммы <a id="lem_old"></a><a id="s2-5-1"></a>


[Второй вариант формирования категорий](#lem_new)

In [25]:
# Соберем множество всех значений из лемматизированного столбца purpose_lem
# Сначала склеим все значения колоноки purpose_lem с помощью ''.join(), 
# разобъем на список из отдельных слов, через split(), а затем оставим 
# только уникальные значения методом set(), все это запишем в words
words_old = set((''.join(data['purpose_lem'])).split())

# Выведем наше множество words
print(words_old)

{'подержать', 'свой', 'подержанный', 'сыграть', 'ремонт', 'сдача', 'дополнительный', 'операция', 'проведение', 'строительство', 'получение', 'жилой', 'свадьба', 'жилье', 'на', 'с', 'профильный', 'коммерческий', 'высокий', 'со', 'покупка', 'сделка', 'собственный', 'образование', 'автомобиль', 'семья', 'для', 'заниматься', 'недвижимость', 'приобретение'}


### Второй вариант формирование списка для целей получения кредита <a id="lem_new"></a><a id="s2-5-2"></a>

Создадим датафрейм **purpose_words** на основе **частоты упоминаний** в лемматизированной колонке **purpose_lem**

In [26]:
# Соберем словарь words из кол-ва уникальных значений
# лемматизированной колонки purpose_lem, посчитанным методом Counter()
words = dict(Counter((''.join(data['purpose_lem'])).split()))


# Создадим датафрейм purpose_words из словаря words, где 
# ключ: значение будут записаны в колонки name и count соответственно
# отсортируем датафрейм по убыванию в count
purpose_words = pd.DataFrame(list(words.items()), 
                             columns=['name', 'count']).sort_values(by='count', 
                                                                    ascending = False).reset_index(drop = True)

# выведем первые 10 строк purpose_words
purpose_words.head(10)

,name,count
0,недвижимость,6351
1,покупка,5897
2,жилье,4460
3,автомобиль,4306
4,образование,4013
5,с,2918
6,операция,2604
7,свадьба,2324
8,свой,2230
9,на,2222


Получили первичный список для категоризации целей кредита. 
* **недвижимость** и **жилье**, но можно будет разделить этот тип на **ком.недвижимость**, используя дополнительное условие проверки по ключу **коммерческий**, а все остальное **жилье** – все-таки это разный вид недвижимости.
* **автомобиль**
* **образование**
* **свадьба**

Также встречается слово **операция** в словосочитаниях с разным видом недвижимости, в том числе и жилья. Давайте проверим, живет ли это слово как отдельная категория - **лечение**

Создадим цикл-проверку на вхождение слова **операция** в сочетании со словани **недвижимость** и **жилье** в лемматизированную строку **purpose_lem** <a id="ret"></a>

In [27]:
# зададим переменную count для счетчика
count = 0

# Посчитаем есть ли строки с 'операция' без 'недвижимость' или 'жилье'
# Запустим цикл длиной датафрейма data
for item in range(len(data)):
    
    # присвоим переменной text строку из ячейки purpose_lem
    text = data.loc[item, 'purpose_lem']
    
    # найдем все строки с ключем 'операция'
    if 'операция' in text: 
        
        # а так же 'недвижимость'
        if 'недвижимость' in text:    
            pass
        
        # и жилье
        elif 'жилье' in text:
            pass
        
        # если таких пересечений, которые были выше - не будет, то запустим счетчик
        else:                        
            count += 1
            
print('Строк со словом "операция", но без "недвижимость" или "жилье":', count)        

Строк со словом "операция", но без "недвижимость" или "жилье": 0


## Вывод

Для лемматизации мы использовали библиотеку **pymystem3**, так как она лучше всего подходит для работы **с предложениями на русском языке**. Тем более нам необходимы слова в нормальной форме. Создали новую колонку **purpose_lem** с лемматизированными строками значений **purpose**.

Выделили категории для целей получения кредита: **ком.недвижимость, жилье, автомобиль, образование, свадьба**

Ключевое слово **операция** в наших данных относится только к **недвижимости** или **жилью**

## Категоризация данных<a id="s2-6"></a>

Категоризируем наши данные в нескольких колонках:
- **children**
- **total_income**
- **purpose**
- **debt**

для **family_status** категории уже имеются

### Категоризируем данные о наличии детей, т.к. нам необходимо будет дать ответ на вопрос: <a id="s2-6-1"></a>
* Есть ли зависимость между наличием детей и возвратом кредита в срок?

то выделим 2 категории: **без детей** и  **с детьми**

In [28]:
# Напишем функцию, которая будет принимать элемент серии датафрейма и
# в зависимости от значения в children будет отдавать значение по условию
def child_group(child_count):
    
    if child_count['children'] == 0: # если 0, то 'без детей'
        return 'без детей'
    
    return 'с детьми' # во всех остальных 'с детьми'
        

# Создадим столбец children_group и запишим в него значения данных через
# метод apply() с функцией child_group и параметром axis=1
data['children_group'] = data.apply(child_group, axis=1)

### Категоризируем данные о ежемесячном доходе <a id="s2-6-2"></a>

Для разделения на категори о ежемесячном доходе, будем использовать квантели по **total_income**. 

Узнаем значения на пересечении **0.25, 0.5 и 0.75** квантели - получим 4 группы. Использовать будем метод **quantile()**

In [29]:
data['total_income'].quantile([0.25, 0.5, 0.75])

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64

Чтобы в итоговых данных категоризация была интуитивно понятна, внесем в названия количественные значения:
* до 107т.р.
* от 107 до 142т.р.
* от 142 до 195т.р.
* от 195т.р.

In [30]:
# Напишем функцию, которая будет принимать элемент серии датафрейма и
# в зависимости от значения в total_income будет отдавать значение по условию
def income_group(row):
    
    if row['total_income'] <= 107000: # доход до 107т.р..
        return 'до 107т.р.'
    
    if row['total_income'] <= 142000: # доход от 107 до 142т.р.
        return 'от 107 до 142т.р.'
    
    if row['total_income'] <= 195000: # доход от 142 до 195т.р. 
        return 'от 142 до 195т.р.'
    
    return 'от 195т.р.' # во всех остальных 'от 195т.р.'
        

# Создадим столбец total_income_group и запишим в него значения данных через
# метод apply() с функцией income_group и параметром axis=1
data['total_income_group'] = data.apply(income_group, axis=1)

### Категоризируем данные о целях получения кредита <a id="s2-6-3"></a>

* **ком.недвижимость**
* **жилье**
* **автомобиль**
* **образование**
* **свадьба**. 

После формирования категорий, проверим таблицу **data на пропуски**.

In [31]:
# Функция, которая принимает один элемент серии и проверяет 
# вхождение ключевого слова в элемент purpose_lem этой серии
# проверка осуществяется по ключевым словам:
# автомобиль, свадьба, жилье, недвижимость и образование
def purpose_grouping(row):
    
    if 'автомобиль' in row['purpose_lem']:
        return 'автомобиль'
    
    if 'свадьба' in row['purpose_lem']:
        return 'свадьба'
    
    if 'образование' in row['purpose_lem']:
        return 'образование'
    
    if 'жилье' or 'недвижимость' in row['purpose_lem']:
        if 'коммерческий' in row['purpose_lem']:
            return 'ком.недвижимость'
        else:
            return 'жилье'


    
# Создадим новый столбец purpose_group, который заполним значениями согласно
# функции purpose_grouping, через метод apply() с параметром axis=1
data['purpose_group'] = data.apply(purpose_grouping, axis=1)

In [32]:
data.isnull().sum()

children              0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
purpose_lem           0
children_group        0
total_income_group    0
purpose_group         0
dtype: int64

### Категоризируем данные о факте несвоевременных выплат кредита<a id="s2-6-4"></a>

* **красавчики** – если, задолженносте не было
* **должники** – в обратном случае 

In [33]:
def debt_grouping(row):
    
    if row['debt'] == 0:
        return 'красавчики'
    
    return 'должники'
    

# Создадим новый столбец purpose_group
data['debt_group'] = data.apply(debt_grouping, axis=1)

# Выведим общую информацию о таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
children              21454 non-null int64
dob_years             21454 non-null int64
education             21454 non-null object
education_id          21454 non-null int64
family_status         21454 non-null object
family_status_id      21454 non-null int64
gender                21454 non-null object
income_type           21454 non-null object
debt                  21454 non-null int64
total_income          21454 non-null int64
purpose               21454 non-null object
purpose_lem           21454 non-null object
children_group        21454 non-null object
total_income_group    21454 non-null object
purpose_group         21454 non-null object
debt_group            21454 non-null object
dtypes: int64(6), object(10)
memory usage: 2.6+ MB


## Вывод

Мы провели категоризацию основных данных в колонках, для дальнейшего исследования:
- **children** выделили 2 группы:
  - **без детей**
  - **с детьми**


- **total_income** выделили 4 групп:
  - **до 107т.р.**
  - **от 107 до 142т.р.**
  - **от 142 до 195т.р.**
  - **от 195т.р.**


Ежемесячный доход разбили на большее кол-во групп, детальнее взяли данные со значениями менее 300 000


- **purpose** – цель получения кредита:
  - **автомобиль**
  - **свадьба**
  - **образование**
  - **жилье**
  - **ком.недвижимость**
  
Данные для **purpose** – цель получения кредита, формировали на основе лемматизации. Разделили жилье и ком.недвижимость.


- **debt** разделили на 2 группы:
  - **должники**
  - **красавчики**

## Шаг 3. Ответьте на вопросы<a id="s3"></a>

Сформируем функцию, которая будет принимать датафрейм и название колонки, по которой будем формировать сводные таблицы для разных вопросов. Передавать будем именно ту колонку, расчет должников с которой и будем сравнивать для каждого из вопросов.

In [34]:
# Функция для подготовки сводной таблицы на основе категоризированной колонки данных
# и информации о задолженности по кредиту с выводом процента ljk;ybrjd
def pivot_func(df, column):
    
    # применим pivot_table для входящего датафрейма в df, а столбец для index=[]
    # заберем из column, группировать всегда будем по debet_group, значения заберем
    # из debt с подсчетом count
    pivot = df.pivot_table(index=[column], columns='debt_group', values='debt', aggfunc='count')
    
    # создадим столбец для расчета процента должников от общего кол-ва группы
    pivot['% должников'] = pivot['должники'] / (pivot['красавчики'] + pivot['должники']) * 100
    
    # для процентов выведем значения с двумя знаками после запятой
    pivot['% должников'] = round(pivot['% должников'], 2)
    
    # отсортируем значения по столбцу с процентами по убыванию
    pivot = pivot.sort_values(by = '% должников', ascending = False)
    
    return pivot

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?<a id="s3-1"></a>

In [35]:
pivot_func(data, 'children_group')

debt_group,должники,красавчики,% должников
children_group,,,
с детьми,678,6685,9.21
без детей,1063,13028,7.54


## Вывод

Доля тех кто **не имеет детей**, более **ответственно подходят** к долговым обязательствам перед ними.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?<a id="s3-2"></a>

In [36]:
pivot_func(data, 'family_status')

debt_group,должники,красавчики,% должников
family_status,,,
не женат / не замужем,274,2536,9.75
гражданский брак,388,3763,9.35
женат / замужем,931,11408,7.55
в разводе,85,1110,7.11
вдовец / вдова,63,896,6.57


## Вывод

Люди, которые находятся в **официальном браке**, **разведены** или **потерявшие вторую половину** несут бОльшую ответственность и осознанность перед банком, нежели люди, которые **не в браке** или находятся **в гражданском браке**

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<a id="s3-3"></a>

In [37]:
pivot_func(data, 'total_income_group')

debt_group,должники,красавчики,% должников
total_income_group,,,
от 142 до 195т.р.,550,5781,8.69
от 107 до 142т.р.,379,4013,8.63
до 107т.р.,423,4884,7.97
от 195т.р.,389,5035,7.17


## Вывод

**Злостные неплательшики** – люди **со средним** доходом. Кординально **противоположны** в своем отношении к банку, **люди** либо **с низким достатком**, либо наоборот – **выше среднего**.

### - Как разные цели кредита влияют на его возврат в срок?<a id="s3-4"></a>

In [38]:
pivot_func(data, 'purpose_group')

debt_group,должники,красавчики,% должников
purpose_group,,,
автомобиль,403,3903,9.36
образование,370,3643,9.22
свадьба,186,2138,8.00
ком.недвижимость,99,1212,7.55
жилье,683,8817,7.19


## Вывод

Люди, которые взяли **кредит на недвижимость** любого вида – **системны в выплатах**. Чего **не скажешь про тех**, кто взял кредит на **авто** и **образование**.

# Шаг 4. Общий вывод<a id="s4"></a>

Пероводя анализ данных, перед нами стояло 4 вопроса:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

На все 4 вопроса можно ответить – **зависимость есть**!

### Кто исправно следит за своими кредитными обязательствами:
 
 - Клиенты, которые **не имеют детей**;
 - Кто находится в **официальном браке**, **официально разведены** или **потерял вторую половину**;
 - У кого доходы либо **ниже среднего**, либо **очень высокие**; 
 - Кто берет кредит на **недвижимость**, будь то жилая или коммерческая.
 
 
### Кто по статистике чаще забывает о банке и своевременных выплатах:

- У кого **есть** хотя бы один **ребенок**;
- Кто **не в браке** или живет **в гражданском браке**;
- Кто имеет **достаток от 100 до 200 т.р.** в месяц;
- Тот, кто берет кредит на **автомобиль** или **образование**.